# NFL Predictor
This Notebook is intended to be used after prior NFL Data has been scraped and the team ELO's and Current starting quarterbacks are accurate

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math
from scipy import stats
import random


In [7]:
nfl_df = pd.read_excel('NFL_Scraper.xlsx', sheet_name = 'NFL_DF')
player_game_rating = pd.read_excel('NFL_Scraper.xlsx', sheet_name = 'QB_Game_Rating')
current_qb_rating = pd.read_excel('NFL_Scraper.xlsx', sheet_name = 'Current_Starting_QBR')
team_elo = pd.read_excel('NFL_Scraper.xlsx', sheet_name = 'Team_ELO')

nfl_df.drop(nfl_df.columns[0], axis = 1, inplace = True)
player_game_rating.drop(player_game_rating.columns[0], axis = 1, inplace = True)
team_elo = team_elo.set_index('Team')
current_qb_rating = current_qb_rating.set_index('Team')

In [8]:
#This cell generates all upcoming NFL games

url_template = "https://www.pro-football-reference.com/years/{year}/games.htm"

current_year = 2021
week = '10' #Change to what week to predict
url = url_template.format(year=current_year)  # get the url
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')
    
column_headers = [th.getText() for th in soup.findAll('thead', limit=1)[0].findAll('th')]    
data_rows = soup.findAll('tbody', limit=1)[0].findAll('tr')[0:]
player_data = [[td.getText() for td in data_rows[i].findAll(['th','td'])] for i in range(len(data_rows))]
nfl_upcoming_games = pd.DataFrame(player_data, columns=column_headers)
nfl_upcoming_games['Year'] = current_year
nfl_upcoming_games = nfl_upcoming_games[(nfl_upcoming_games.Day !='Day') & (nfl_upcoming_games.Week == week)]
    

In [9]:
def visiting_elo_adjustment(game):
    elo_adjustment = 0
    elo_adjustment = elo_adjustment + current_qb_rating.loc[game['Winner/tie']]['Current QBR']
    
    return elo_adjustment

In [10]:
def home_elo_adjustment(game):
    elo_adjustment = 0
    elo_adjustment = elo_adjustment + 30 + current_qb_rating.loc[game['Loser/tie']]['Current QBR']
    
    return elo_adjustment

In [11]:
def elo_probability_calculator(game):
    #Elo System Design inspired by: https://fivethirtyeight.com/methodology/how-our-nfl-predictions-work/
    #The name of the team references the team dictionary, which provides the reference value for the team's elo value.
    #This was included to account for name changes of teams. For the sake of the ELO measurement, a team that switches location will retain the same ELO
    visiting_team_elo = team_elo.loc[game['Winner/tie']]['elo']
    home_team_elo = team_elo.loc[game['Loser/tie']]['elo']
    
    #ELO Adjustments based on different conditions
    visiting_team_adjustment = visiting_elo_adjustment(game)
    home_team_adjustment = home_elo_adjustment(game)
    
    elo_diff = (home_team_elo + home_team_adjustment) - (visiting_team_elo + visiting_team_adjustment)
    
    #Probability of the winning team winning, before ELO adjustment
    prob_win = 1/((10**(-1*elo_diff/400)) + 1)
   
    return (prob_win, elo_diff)
    

In [12]:
nfl_upcoming_games

,Week,Day,Date,Time,Winner/tie,,Loser/tie,,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Year
145,10,Thu,2021-11-11,8:20PM,Miami Dolphins,,Baltimore Ravens,boxscore,22,10,350,0,304,2,2021
146,10,Sun,2021-11-14,1:00PM,Detroit Lions,@,Pittsburgh Steelers,preview,,,,,,,2021
147,10,Sun,2021-11-14,1:00PM,Atlanta Falcons,@,Dallas Cowboys,preview,,,,,,,2021
148,10,Sun,2021-11-14,1:00PM,Buffalo Bills,@,New York Jets,preview,,,,,,,2021
149,10,Sun,2021-11-14,1:00PM,Cleveland Browns,@,New England Patriots,preview,,,,,,,2021
150,10,Sun,2021-11-14,1:00PM,New Orleans Saints,@,Tennessee Titans,preview,,,,,,,2021
151,10,Sun,2021-11-14,1:00PM,Tampa Bay Buccaneers,@,Washington Football Team,preview,,,,,,,2021
152,10,Sun,2021-11-14,1:00PM,Jacksonville Jaguars,@,Indianapolis Colts,preview,,,,,,,2021
153,10,Sun,2021-11-14,4:05PM,Carolina Panthers,@,Arizona Cardinals,preview,,,,,,,2021
154,10,Sun,2021-11-14,4:05PM,Minnesota Vikings,@,Los Angeles Chargers,preview,,,,,,,2021


In [13]:
nfl_upcoming_games['Visiting_Win_Probability'] = 1-nfl_upcoming_games.apply(lambda row: elo_probability_calculator(row)[0], axis = 1)
nfl_upcoming_games['Home_Win_Probability'] = 1-nfl_upcoming_games['Visiting_Win_Probability'] 
nfl_upcoming_games['Elo_Difference'] = nfl_upcoming_games.apply(lambda row: elo_probability_calculator(row)[1], axis = 1)

In [14]:
nfl_upcoming_games.drop(['PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL', ''], axis = 1, inplace = True)

In [15]:
nfl_upcoming_games.rename(columns = {'Winner/tie': 'Visiting Team', 'Loser/tie': 'Home Team'}, inplace = True)

In [16]:
with pd.ExcelWriter('NFL_Predictions2020.xlsx',
                    mode='a', engine ="openpyxl") as writer:  
     nfl_upcoming_games.to_excel(writer, sheet_name='2021Week' + week)
writer.save()   
        

ValueError: I/O operation on closed file

In [17]:
nfl_upcoming_games

,Week,Day,Date,Time,Visiting Team,Home Team,Year,Visiting_Win_Probability,Home_Win_Probability,Elo_Difference
145,10,Thu,2021-11-11,8:20PM,Miami Dolphins,Baltimore Ravens,2021,0.166541,0.833459,279.744893
146,10,Sun,2021-11-14,1:00PM,Detroit Lions,Pittsburgh Steelers,2021,0.097005,0.902995,387.555745
147,10,Sun,2021-11-14,1:00PM,Atlanta Falcons,Dallas Cowboys,2021,0.291465,0.708535,154.310079
148,10,Sun,2021-11-14,1:00PM,Buffalo Bills,New York Jets,2021,0.909685,0.090315,-401.251760
149,10,Sun,2021-11-14,1:00PM,Cleveland Browns,New England Patriots,2021,0.442862,0.557138,39.877483
150,10,Sun,2021-11-14,1:00PM,New Orleans Saints,Tennessee Titans,2021,0.427214,0.572786,50.938452
151,10,Sun,2021-11-14,1:00PM,Tampa Bay Buccaneers,Washington Football Team,2021,0.878977,0.121023,-344.444057
152,10,Sun,2021-11-14,1:00PM,Jacksonville Jaguars,Indianapolis Colts,2021,0.114711,0.885289,354.992799
153,10,Sun,2021-11-14,4:05PM,Carolina Panthers,Arizona Cardinals,2021,0.109880,0.890120,363.411303
154,10,Sun,2021-11-14,4:05PM,Minnesota Vikings,Los Angeles Chargers,2021,0.373948,0.626052,89.519636
